<a href="https://colab.research.google.com/github/amrutha2413/Telescope/blob/main/Telescope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install transformers torch --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 768.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 509.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 575.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print("Device:", device)


Loading AI model...


Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Device: cpu


In [ ]:
def top_k_next_tokens(current_text, k=5):
    input_ids = tokenizer.encode(current_text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits  # [batch, seq_len, vocab]

    next_token_logits = logits[0, -1, :]                 # logits for next token
    probs = torch.softmax(next_token_logits, dim=0)      # convert to probabilities

    top_probs, top_indices = torch.topk(probs, k)
    results = []
    for prob, idx in zip(top_probs, top_indices):
        token_str = tokenizer.decode([idx.item()])
        results.append((token_str, prob.item(), idx.item()))
    return results


In [ ]:
import random

def generate_token_by_token(
    prompt,
    steps=5,
    k=5,
    pick_strategy="greedy",  # "greedy" or "top_k"
    seed=42
):
    """
    greedy: always pick the most probable next token
    top_k: sample randomly among the top-k tokens using their probabilities
    """
    torch.manual_seed(seed)
    random.seed(seed)

    current_text = prompt
    print(f"\nStarting text: {repr(prompt)}")
    print(f"steps={steps}, k={k}, pick_strategy={pick_strategy}\n")

    for step in range(steps):
        print(f"--- Step {step + 1} ---")
        print(f"Current: {repr(current_text)}")

        candidates = top_k_next_tokens(current_text, k=k)

        print(f"Top {k} next-token predictions:")
        for i, (tok, prob, tid) in enumerate(candidates, start=1):
            print(f"  {i}. {repr(tok)} ({prob*100:.2f}%)  token_id={tid}")

        # choose next token
        if pick_strategy == "greedy":
            chosen_tok, chosen_prob, chosen_id = candidates[0]
        elif pick_strategy == "top_k":
            # sample among top-k using normalized probs
            probs = torch.tensor([p for _, p, _ in candidates], dtype=torch.float)
            probs = probs / probs.sum()
            choice_idx = torch.multinomial(probs, num_samples=1).item()
            chosen_tok, chosen_prob, chosen_id = candidates[choice_idx]
        else:
            raise ValueError("pick_strategy must be 'greedy' or 'top_k'")

        current_text += chosen_tok
        print(f"✓ Chosen: {repr(chosen_tok)} (from token_id={chosen_id})")
        print(f"New text: {repr(current_text)}\n")

    print("FINAL:", repr(current_text))
    return current_text


In [ ]:
text = "The weather today is "
generate_token_by_token(prompt=text, steps=5, k=5, pick_strategy="greedy", seed=42)



Starting text: 'The weather today is '
steps=5, k=5, pick_strategy=greedy

--- Step 1 ---
Current: 'The weather today is '
Top 5 next-token predictions:
  1. 'iced' (45.60%)  token_id=3711
  2. '\xa0' (18.60%)  token_id=1849
  3. '!!!' (2.48%)  token_id=10185
  4. '�' (2.00%)  token_id=135
  5. '????' (1.78%)  token_id=9805
✓ Chosen: 'iced' (from token_id=3711)
New text: 'The weather today is iced'

--- Step 2 ---
Current: 'The weather today is iced'
Top 5 next-token predictions:
  1. ' up' (13.14%)  token_id=510
  2. ' with' (11.31%)  token_id=351
  3. ' and' (8.39%)  token_id=290
  4. '.' (8.00%)  token_id=13
  5. ',' (7.97%)  token_id=11
✓ Chosen: ' up' (from token_id=510)
New text: 'The weather today is iced up'

--- Step 3 ---
Current: 'The weather today is iced up'
Top 5 next-token predictions:
  1. ',' (15.70%)  token_id=11
  2. ' and' (14.88%)  token_id=290
  3. '.' (10.33%)  token_id=13
  4. ' by' (6.93%)  token_id=416
  5. ' to' (4.85%)  token_id=284
✓ Chosen: ',' (from toke

'The weather today is iced up, and the'

In [ ]:
# Change 1: steps=30 (keep everything else the same)
text = "The weather today is "
generate_token_by_token(prompt=text, steps=30, k=5, pick_strategy="top_k", seed=42)



Starting text: 'The weather today is '
steps=30, k=5, pick_strategy=top_k

--- Step 1 ---
Current: 'The weather today is '
Top 5 next-token predictions:
  1. 'iced' (45.60%)  token_id=3711
  2. '\xa0' (18.60%)  token_id=1849
  3. '!!!' (2.48%)  token_id=10185
  4. '�' (2.00%)  token_id=135
  5. '????' (1.78%)  token_id=9805
✓ Chosen: 'iced' (from token_id=3711)
New text: 'The weather today is iced'

--- Step 2 ---
Current: 'The weather today is iced'
Top 5 next-token predictions:
  1. ' up' (13.14%)  token_id=510
  2. ' with' (11.31%)  token_id=351
  3. ' and' (8.39%)  token_id=290
  4. '.' (8.00%)  token_id=13
  5. ',' (7.97%)  token_id=11
✓ Chosen: '.' (from token_id=13)
New text: 'The weather today is iced.'

--- Step 3 ---
Current: 'The weather today is iced.'
Top 5 next-token predictions:
  1. '�' (21.13%)  token_id=447
  2. ' I' (8.85%)  token_id=314
  3. '\n' (5.88%)  token_id=198
  4. ' It' (5.63%)  token_id=632
  5. ' The' (4.90%)  token_id=383
✓ Chosen: '�' (from token_id=44

"The weather today is iced.�<|endoftext|>The first of three major US presidential elections in the country has been called for in the United States to be a 'progressive'"

In [ ]:
# Change 2: k=3 (keep steps=30)
text = "The weather today is "
generate_token_by_token(prompt=text, steps=30, k=3, pick_strategy="greedy", seed=42)



Starting text: 'The weather today is '
steps=30, k=3, pick_strategy=greedy

--- Step 1 ---
Current: 'The weather today is '
Top 3 next-token predictions:
  1. 'iced' (45.60%)  token_id=3711
  2. '\xa0' (18.60%)  token_id=1849
  3. '!!!' (2.48%)  token_id=10185
✓ Chosen: 'iced' (from token_id=3711)
New text: 'The weather today is iced'

--- Step 2 ---
Current: 'The weather today is iced'
Top 3 next-token predictions:
  1. ' up' (13.14%)  token_id=510
  2. ' with' (11.31%)  token_id=351
  3. ' and' (8.39%)  token_id=290
✓ Chosen: ' up' (from token_id=510)
New text: 'The weather today is iced up'

--- Step 3 ---
Current: 'The weather today is iced up'
Top 3 next-token predictions:
  1. ',' (15.70%)  token_id=11
  2. ' and' (14.88%)  token_id=290
  3. '.' (10.33%)  token_id=13
✓ Chosen: ',' (from token_id=11)
New text: 'The weather today is iced up,'

--- Step 4 ---
Current: 'The weather today is iced up,'
Top 3 next-token predictions:
  1. ' and' (17.85%)  token_id=290
  2. '�' (14.88%) 

'The weather today is iced up, and the weather is getting better.�\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

In [ ]:
# Change 3: new prompt (keep steps=30, k=3)
text = "The moon was shining bright at night"
generate_token_by_token(prompt=text, steps=30, k=3, pick_strategy="greedy", seed=42)



Starting text: 'The moon was shining bright at night'
steps=30, k=3, pick_strategy=greedy

--- Step 1 ---
Current: 'The moon was shining bright at night'
Top 3 next-token predictions:
  1. ',' (28.29%)  token_id=11
  2. '.' (18.84%)  token_id=13
  3. ' and' (9.60%)  token_id=290
✓ Chosen: ',' (from token_id=11)
New text: 'The moon was shining bright at night,'

--- Step 2 ---
Current: 'The moon was shining bright at night,'
Top 3 next-token predictions:
  1. ' and' (31.03%)  token_id=290
  2. ' but' (11.19%)  token_id=475
  3. ' the' (3.95%)  token_id=262
✓ Chosen: ' and' (from token_id=290)
New text: 'The moon was shining bright at night, and'

--- Step 3 ---
Current: 'The moon was shining bright at night, and'
Top 3 next-token predictions:
  1. ' the' (19.38%)  token_id=262
  2. ' it' (5.54%)  token_id=340
  3. ' when' (3.39%)  token_id=618
✓ Chosen: ' the' (from token_id=262)
New text: 'The moon was shining bright at night, and the'

--- Step 4 ---
Current: 'The moon was shining br

'The moon was shining bright at night, and the moon was shining bright at night, and the moon was shining bright at night, and the moon was shining bright at night, and the'

In [ ]:
# Change 4: longer prompt (keep steps=30, k=3)
text = (
    "The moon was shining bright at night as I walked home quietly, "
    "hearing distant cars and feeling the cold air on my face. "
)
generate_token_by_token(prompt=text, steps=30, k=3, pick_strategy="greedy", seed=42)



Starting text: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. '
steps=30, k=3, pick_strategy=greedy

--- Step 1 ---
Current: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. '
Top 3 next-token predictions:
  1. '\xa0' (41.96%)  token_id=1849
  2. '~~' (5.73%)  token_id=4907
  3. '________________' (4.19%)  token_id=4841
✓ Chosen: '\xa0' (from token_id=1849)
New text: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. \xa0'

--- Step 2 ---
Current: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. \xa0'
Top 3 next-token predictions:
  1. '\n' (47.09%)  token_id=198
  2. ' I' (6.09%)  token_id=314
  3. ' The' (4.24%)  token_id=383
✓ Chosen: '\n' (from token_id=198)
New text: 'The moon was shining brig

'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. \xa0\nI was so excited to see the moon. \xa0\nI was so excited to see the moon. \xa0\nI was so excited to see'

In [ ]:
# Change 5a: greedy (keep steps=30, k=3)
text = (
    "The moon was shining bright at night as I walked home quietly, "
    "hearing distant cars and feeling the cold air on my face. "
)
generate_token_by_token(prompt=text, steps=30, k=3, pick_strategy="greedy", seed=42)



Starting text: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. '
steps=30, k=3, pick_strategy=greedy

--- Step 1 ---
Current: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. '
Top 3 next-token predictions:
  1. '\xa0' (41.96%)  token_id=1849
  2. '~~' (5.73%)  token_id=4907
  3. '________________' (4.19%)  token_id=4841
✓ Chosen: '\xa0' (from token_id=1849)
New text: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. \xa0'

--- Step 2 ---
Current: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. \xa0'
Top 3 next-token predictions:
  1. '\n' (47.09%)  token_id=198
  2. ' I' (6.09%)  token_id=314
  3. ' The' (4.24%)  token_id=383
✓ Chosen: '\n' (from token_id=198)
New text: 'The moon was shining brig

'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. \xa0\nI was so excited to see the moon. \xa0\nI was so excited to see the moon. \xa0\nI was so excited to see'

In [ ]:
# Change 5b: top-k sampling (keep steps=30, k=3)
text = (
    "The moon was shining bright at night as I walked home quietly, "
    "hearing distant cars and feeling the cold air on my face. "
)
generate_token_by_token(prompt=text, steps=30, k=3, pick_strategy="top_k", seed=42)



Starting text: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. '
steps=30, k=3, pick_strategy=top_k

--- Step 1 ---
Current: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. '
Top 3 next-token predictions:
  1. '\xa0' (41.96%)  token_id=1849
  2. '~~' (5.73%)  token_id=4907
  3. '________________' (4.19%)  token_id=4841
✓ Chosen: '\xa0' (from token_id=1849)
New text: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. \xa0'

--- Step 2 ---
Current: 'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. \xa0'
Top 3 next-token predictions:
  1. '\n' (47.09%)  token_id=198
  2. ' I' (6.09%)  token_id=314
  3. ' The' (4.24%)  token_id=383
✓ Chosen: '\n' (from token_id=198)
New text: 'The moon was shining brigh

'The moon was shining bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. \xa0\nThe moon was bright at night as I walked home quietly, hearing distant cars and feeling the cold air on my face. \xa0\nThe sun'